# Extraindo empresas

## Extraindo todas empresas do Brasil da categoria BESST

In [ ]:
import requests
import yfinance as yf
import pandas as pd
import os

# --- PASSO 1: CONFIGURAÇÃO DOS SETORES-ALVO ---
setores_alvo = [
    "Finance",
    "Utilities",
    "Communications",
    "Industrial Services"
]

# --- FUNÇÃO PRINCIPAL COM A LÓGICA FINAL ---
def scanner_final_com_tipo(setores):
    print("Iniciando scanner final focado (filtrando por tipo 'stock' e 'fund')...")
    try:
        url = "https://brapi.dev/api/quote/list"
        lista_completa_brapi = requests.get(url).json().get('stocks', [])
    except Exception as e:
        print(f"❌ Erro fatal ao buscar a lista da brapi: {e}")
        return {}

    print("Mapeando ativos dos setores alvo...")
    tickers_para_analisar = {}
    for stock in lista_completa_brapi:
        setor_brapi = stock.get('sector')
        tipo_ativo = stock.get('type')
        if setor_brapi in setores and tipo_ativo in ['stock', 'fund']:
            ticker = f"{stock.get('stock')}.SA"
            tickers_para_analisar[ticker] = {
                'setor': setor_brapi,
                'logo': stock.get('logo'),
                'tipo': tipo_ativo 
            }
    
    total = len(tickers_para_analisar)
    print(f"✅ Mapeamento concluído. {total} ativos dos tipos 'stock' e 'fund' serão analisados.")

    dados_finais = {}
    print("\nIniciando coleta detalhada de dados no yfinance...")
    for i, (ticker, brapi_data) in enumerate(tickers_para_analisar.items()):
        try:
            print(f"Analisando [{i+1}/{total}]: {ticker}...")
            stock = yf.Ticker(ticker)
            info = stock.info
            
            if info.get("country") != "Brazil":
                print(f"  -> DESCARTADO: {ticker} não é do Brasil.")
                continue
            
            dados_finais[ticker] = {
                "Empresa": info.get("longName", "N/A"),
                "Setor (brapi)": brapi_data['setor'],
                "Subsetor (yfinance)": info.get("industry", "N/A"),
                "País": info.get("country", "N/A"),
                "Tipo": brapi_data['tipo'],
                "Market Cap": info.get("marketCap"),
                "Logo": brapi_data['logo'],
            }
            print(f"  -> ✅ INCLUÍDO: {ticker}")

        except Exception as e:
            print(f"  -> ERRO: Falha ao processar {ticker}. Erro: {e}")
            continue
                
    return dados_finais

# --- EXECUÇÃO E APRESENTAÇÃO DOS RESULTADOS ---
resultados = scanner_final_com_tipo(setores_alvo)

if resultados:
    print("\n✅ Análise finalizada.")
    df_resultado = pd.DataFrame.from_dict(resultados, orient="index")
    
    df_resultado.reset_index(inplace=True)
    df_resultado.rename(columns={'index': 'Ticker'}, inplace=True)
    
    if "Market Cap" in df_resultado.columns:
         df_resultado["Market Cap Num"] = pd.to_numeric(df_resultado["Market Cap"], errors='coerce')
         df_resultado["Market Cap"] = df_resultado["Market Cap Num"].apply(
             lambda x: f"R$ {x/1_000_000_000:.2f} Bi" if pd.notna(x) and x > 0 else "N/A"
         )

    df_ordenado = df_resultado.sort_values(by=['Setor (brapi)', 'Market Cap Num'], ascending=[True, False])
    
    colunas_finais = ['Ticker', 'Empresa', 'Setor (brapi)', 'Subsetor (yfinance)', 'País', 'Tipo', 'Market Cap', 'Logo']
    df_final = df_ordenado[colunas_finais]
    
    print("\n--- Relatório Final (Ações e Fundos) ---")
    print(df_final.to_string(index=False))

    # --- SALVANDO O ARQUIVO NO CAMINHO ESPECÍFICO ---
    try:
        output_folder = r"E:\finance-manager\data"
        output_filename = "scanner_acoes_e_fundos_filtrado.csv"
        full_path = os.path.join(output_folder, output_filename)
        
        os.makedirs(output_folder, exist_ok=True)
        
        df_final.to_csv(full_path, index=False)
        print(f"\n✅ Resultados salvos com sucesso no caminho: {full_path}")
    except Exception as e:
        print(f"\n❌ Erro ao salvar o arquivo CSV: {e}")
else:
    print("\nNenhuma empresa foi encontrada para os critérios especificados.")

In [ ]:
# import pandas as pd

# df_dados2 = pd.read_excel(r'E:\Github\finance-manager\datasets\scanner_acoes_filtrado.xlsx')

# print(df_dados2)

# Transformação

In [1]:
# transform.py (Versão Completa e Robusta para Diagnóstico)
import yfinance as yf
import pandas as pd
import os
from datetime import datetime

def get_market_sentiment_and_details(ticker_obj: yf.Ticker) -> dict:
    """
    Usa 'recommendations_summary' para maior robustez.
    Calcula o sentimento de mercado (0-100) e retorna os contadores individuais.
    """
    # Dicionário padrão para garantir que as colunas sempre existam
    sentiment_data = {
        'Sentimento Gauge': 50.0, 'Strong Buy': 0, 'Buy': 0,
        'Hold': 0, 'Sell': 0, 'Strong Sell': 0
    }
    try:
        summary = ticker_obj.recommendations_summary
        # Verifica se o sumário está vazio ou não foi retornado
        if summary is None or summary.empty:
            return sentiment_data

        # Pega a linha mais recente do sumário
        rec_counts = summary.iloc[-1]
        
        strong_buy = int(rec_counts.get('strongBuy', 0))
        buy = int(rec_counts.get('buy', 0))
        hold = int(rec_counts.get('hold', 0))
        sell = int(rec_counts.get('sell', 0))
        strong_sell = int(rec_counts.get('strongSell', 0))

        sentiment_data.update({
            'Strong Buy': strong_buy, 'Buy': buy, 'Hold': hold,
            'Sell': sell, 'Strong Sell': strong_sell
        })

        # Calcula o score ponderado das recomendações
        weighted_score = (strong_buy * 2) + (buy * 1) + (hold * 0) + (sell * -1) + (strong_sell * -2)
        total_recommendations = strong_buy + buy + hold + sell + strong_sell

        if total_recommendations == 0:
            return sentiment_data # Retorna neutro se não houver recomendações

        # Normaliza o score médio de [-2, 2] para a escala de [0, 100]
        avg_score = weighted_score / total_recommendations
        normalized_sentiment = ((avg_score + 2) / 4) * 100
        sentiment_data['Sentimento Gauge'] = normalized_sentiment
        
        return sentiment_data

    except Exception as e:
        print(f" (Aviso: Falha ao buscar sentimento - {e}) ", end="")
        # Em caso de qualquer erro, retorna os dados neutros/padrão
        return sentiment_data

def fetch_stock_data(ticker: str, info_original: pd.Series) -> dict:
    """
    Busca e processa os dados de um único ticker.
    """
    stock = yf.Ticker(ticker)
    info = stock.info
    history = stock.history(period="5y")

    # --- Cálculos de Indicadores ---
    dy_12m = info.get("trailingAnnualDividendYield", 0.0) * 100 if info.get("trailingAnnualDividendYield") is not None else 0.0
    dy_5y = info.get("fiveYearAvgDividendYield", 0.0) * 100 if info.get("fiveYearAvgDividendYield") is not None else 0.0

    growth_price = 0.0
    if not history.empty and len(history['Close']) > 1 and history['Close'].iloc[0] > 0:
        growth_price = ((history['Close'].iloc[-1] / history['Close'].iloc[0]) - 1) * 100

    payout_ratio = info.get("payoutRatio", 0.0) * 100 if info.get("payoutRatio") is not None else 0.0

    # --- Coleta de Sentimento ---
    sentiment_info = get_market_sentiment_and_details(stock)

    # --- Dicionário de Dados ---
    dados = {
        "Empresa": info_original.get('Empresa', 'N/A'),
        "Setor (brapi)": info_original.get('Setor (brapi)', 'N/A'),
        "Tipo": info_original.get('Tipo', 'N/A'),
        "Market Cap": info_original.get('Market Cap', 0),
        "Logo": info_original.get('Logo', 'N/A'),
        "Preço Atual": info.get("currentPrice", 0.0),
        "P/L": info.get("trailingPE"),
        "P/VP": info.get("priceToBook"),
        "DY (Taxa 12m, %)": dy_12m,
        "DY 5 Anos Média (%)": dy_5y,
        "Último Dividendo": info.get("lastDividendValue"),
        "Data Últ. Div.": pd.to_datetime(info.get("lastDividendDate"), unit='s', errors='coerce'),
        "Data Ex-Div.": pd.to_datetime(info.get("exDividendDate"), unit='s', errors='coerce'),
        "Payout Ratio (%)": payout_ratio,
        "Crescimento Preço (%)": growth_price,
        "ROE (%)": info.get("returnOnEquity", 0.0) * 100 if info.get("returnOnEquity") is not None else 0.0,
        "Dívida Total": info.get('totalDebt'),
        "EBITDA": info.get('ebitda'),
    }
    
    # Adiciona os dados de sentimento ao dicionário principal
    dados.update(sentiment_info)

    if dados.get("EBITDA") and dados["EBITDA"] != 0 and dados.get("Dívida Total"):
        dados["Dívida/EBITDA"] = dados["Dívida Total"] / dados["EBITDA"]
    else:
        dados["Dívida/EBITDA"] = None

    return dados

def main():
    """
    Função principal para executar o processo de coleta e salvamento de dados.
    """
    # --- CONFIGURAÇÃO DOS CAMINHOS ---
    # Caminho do arquivo de entrada com a lista de tickers
    arquivo_input = r"E:\Github\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv"
    
    # Caminho da pasta de saída
    output_folder = r"E:\Github\finance-manager\data"
    output_filename = "relatorio_analise_b3.csv"
    # Usa os.path.join para criar o caminho completo de forma segura
    output_path = os.path.join(output_folder, output_filename)

    # --- PASSO 1: LER ARQUIVO DE ENTRADA ---
    try:
        print(f"Lendo a lista de ativos de '{arquivo_input}'...")
        df_input = pd.read_csv(arquivo_input)
        tickers_para_analisar = df_input['Ticker'].tolist()
        print(f"✅ {len(tickers_para_analisar)} ativos encontrados para análise.")
    except FileNotFoundError:
        print(f"❌ ERRO CRÍTICO: Arquivo de entrada '{arquivo_input}' não encontrado. O processo não pode continuar.")
        return
    except Exception as e:
        print(f"❌ ERRO CRÍTICO ao ler o arquivo de entrada: {e}")
        return

    # --- PASSO 2: COLETAR DADOS DA API ---
    dados_detalhados = {}
    erros = []
    print("\nIniciando coleta de dados via yfinance...")
    for ticker in tickers_para_analisar:
        print(f"Processando: {ticker}...", end="")
        try:
            info_original = df_input[df_input['Ticker'] == ticker].iloc[0]
            dados_detalhados[ticker] = fetch_stock_data(ticker, info_original)
            print(" ✅")
        except Exception as e:
            print(f" ❌ (Erro: {e})")
            erros.append(ticker)

    # --- PASSO 3: VERIFICAR E ORGANIZAR DADOS ---
    if not dados_detalhados:
        print("\n❌ NENHUM DADO FOI COLETADO COM SUCESSO. O arquivo CSV não será gerado.")
        return

    print("\nOrganizando dados coletados...")
    df_resultado = pd.DataFrame.from_dict(dados_detalhados, orient="index")
    
    # Garante que as colunas de data sejam do tipo datetime
    for col in ['Data Últ. Div.', 'Data Ex-Div.']:
        df_resultado[col] = pd.to_datetime(df_resultado[col], errors='coerce')
    
    # Preenche valores numéricos ausentes com 0 para evitar erros no app
    df_resultado.fillna(0, inplace=True)
    
    # --- DIAGNÓSTICO: MOSTRAR PRÉVIA DOS DADOS ---
    print("\n--- Prévia dos dados a serem salvos ---")
    print(df_resultado.head())
    print("----------------------------------------")
    
    # --- PASSO 4: SALVAR O ARQUIVO CSV ---
    try:
        print(f"\nTentando salvar o relatório em '{output_path}'...")
        # Garante que a pasta de destino exista. Se não existir, ela será criada.
        os.makedirs(output_folder, exist_ok=True)
        
        # Salva o DataFrame no arquivo CSV
        df_resultado.to_csv(output_path)
        
        print(f"\n✅ SUCESSO! Relatório salvo com sucesso em: {output_path}")
        
    except PermissionError:
        print(f"❌ ERRO DE PERMISSÃO: O script não tem permissão para escrever na pasta '{output_folder}'. Tente executar como administrador ou verifique as permissões da pasta.")
    except Exception as e:
        print(f"❌ ERRO INESPERADO AO SALVAR O ARQUIVO: {e}")

    if erros:
        print(f"\n⚠️ Atenção: {len(erros)} ativos falharam durante a coleta. Ex: {erros[:5]}")

    print(f"\nProcesso concluído às {datetime.now().strftime('%H:%M:%S')}")

if __name__ == "__main__":
    main()


Lendo a lista de ativos de 'E:\Github\finance-manager\data\scanner_acoes_e_fundos_filtrado.csv'...
✅ 145 ativos encontrados para análise.

Iniciando coleta de dados via yfinance...
Processando: VIVT3.SA... ✅
Processando: TIMS3.SA... ✅
Processando: FIQE3.SA... ✅
Processando: BRST3.SA... ✅
Processando: DESK3.SA... ✅
Processando: TELB4.SA... ✅
Processando: TELB3.SA... ✅
Processando: OIBR3.SA... ✅
Processando: OIBR4.SA... ✅
Processando: ITUB3.SA... ✅
Processando: ITUB4.SA... ✅
Processando: BPAC3.SA... ✅
Processando: BPAC5.SA... ✅
Processando: BBDC3.SA... ✅
Processando: BBDC4.SA... ✅
Processando: BPAC11.SA... ✅
Processando: SANB11.SA... ✅
Processando: BBAS3.SA... ✅
Processando: ITSA4.SA... ✅
Processando: ITSA3.SA... ✅
Processando: SANB3.SA... ✅
Processando: SANB4.SA... ✅
Processando: B3SA3.SA... ✅
Processando: BBSE3.SA... ✅
Processando: CXSE3.SA... ✅
Processando: RENT3.SA... ✅
Processando: PSSA3.SA... ✅
Processando: MULT3.SA... ✅
Processando: ALOS3.SA... ✅
Processando: BPAN4.SA... ✅
Process

C:\Users\Atlas Core\AppData\Local\Temp\ipykernel_10664\2666458682.py:163: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df_resultado.fillna(0, inplace=True)


In [ ]:
# import pandas as pd

# df_dados = pd.read_excel(r'E:\Github\finance-manager\datasets\relatorio_completo_b3.xlsx')

# print(df_dados.head(2))